In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_csv("../data/candles/EUR_JPY_H1.csv")

In [4]:
df_raw.shape

(67804, 15)

In [5]:
df_an = df_raw.copy()  # df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.shape

(67804, 15)

In [7]:
df_an = RSI(df_an)

In [8]:
df_an.tail()

,Unnamed: 0,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
67799,67799,2023-11-29 19:00:00+00:00,6019,161.667,161.766,161.654,161.716,161.655,161.756,161.644,161.708,161.679,161.775,161.662,161.724,41.698113
67800,67800,2023-11-29 20:00:00+00:00,4639,161.716,161.744,161.618,161.628,161.707,161.734,161.608,161.618,161.724,161.754,161.626,161.638,37.742504
67801,67801,2023-11-29 21:00:00+00:00,2365,161.628,161.652,161.490,161.506,161.618,161.642,161.476,161.495,161.639,161.663,161.500,161.517,35.401158
67802,67802,2023-11-29 22:00:00+00:00,1904,161.512,161.545,161.293,161.379,161.502,161.502,161.218,161.362,161.522,161.620,161.329,161.396,32.108027
67803,67803,2023-11-29 23:00:00+00:00,3326,161.376,161.424,161.128,161.375,161.361,161.413,161.113,161.364,161.392,161.435,161.141,161.386,26.987952


In [9]:
df_an = apply_patterns(df_an)

In [10]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [11]:
df_an.columns

Index(['Unnamed: 0', 'time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
       'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c',
       'RSI_14', 'body_lower', 'body_upper', 'body_bottom_percentage',
       'body_top_percentage', 'body_percentage', 'direction', 'body_size',
       'low_change', 'high_change', 'body_size_change', 'mid_point',
       'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_percentage_prev', 'body_percentage_prev_2',
       'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING',
       'TWEEZER_TOP', 'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'DOJI',
       'HAMMER', 'INVERTED_HAMMER', 'DARK_CLOUD_COVER', 'PIERCING_PATTERN',
       'BULLISH_ENGULFING', 'BULLISH_HARAMI', 'BEARISH_ENGULFING',
       'BEARISH_HARAMI', 'THREE_WHITE_SOLDIERS', 'THREE_BLACK_CROWS',
       'BULLISH_ABANDONED_BABY', 'BEARISH_ABANDONED_BABY', 'BULLISH_TRI_STAR',
       'EMA_200'],
      dtype='object'

In [12]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c',
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [13]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [14]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,ENGULFING,direction,EMA_200,RSI_14
0,2013-01-17 04:00:00+00:00,117.234,117.339,117.012,117.332,117.225,117.331,117.002,117.320,117.242,117.350,117.020,117.343,False,1,117.499932,50.479233
1,2013-01-17 05:00:00+00:00,117.332,117.856,117.196,117.838,117.321,117.846,117.187,117.828,117.343,117.865,117.203,117.848,False,1,117.503816,52.801462
2,2013-01-17 06:00:00+00:00,117.838,117.994,117.704,117.876,117.827,117.984,117.693,117.866,117.849,118.003,117.715,117.887,False,1,117.508086,49.113592
3,2013-01-17 07:00:00+00:00,117.878,118.065,117.620,117.777,117.869,118.055,117.610,117.767,117.887,118.075,117.629,117.787,True,-1,117.511166,53.562478
4,2013-01-17 08:00:00+00:00,117.774,118.610,117.761,118.545,117.764,118.602,117.751,118.536,117.784,118.619,117.771,118.554,True,1,117.522991,60.945498


In [15]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0


def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [16]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [17]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    62267
 1     2894
-1     2444
Name: count, dtype: int64

In [18]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5


def apply_take_profit(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return (row.ask_c - row.ask_o) * PROFIT_FACTOR + row.ask_c
        else:
            return (row.bid_c - row.bid_o) * PROFIT_FACTOR + row.bid_c
    else:
        return 0.0


def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return row.ask_o
        else:
            return row.bid_o
    else:
        return 0.0

In [19]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [20]:
df_slim[df_slim.SIGNAL == SELL].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
86,2013-01-22 15:00:00+00:00,118.298,118.298,117.608,117.640,118.278,118.282,117.595,117.630,118.319,118.321,117.616,117.650,True,-1,118.654612,33.890747,-1,116.6580,118.278
94,2013-01-22 23:00:00+00:00,118.245,118.298,117.718,117.848,118.230,118.287,117.704,117.837,118.260,118.308,117.728,117.860,True,-1,118.607491,48.481397,-1,117.2475,118.230
98,2013-01-23 03:00:00+00:00,118.100,118.115,117.903,117.904,118.092,118.109,117.896,117.897,118.107,118.122,117.910,117.912,True,-1,118.584370,41.711957,-1,117.6045,118.092
110,2013-01-23 15:00:00+00:00,118.127,118.294,117.531,117.608,118.117,118.283,117.520,117.597,118.137,118.304,117.542,117.620,True,-1,118.482394,40.605590,-1,116.8170,118.117
408,2013-02-08 20:00:00+00:00,124.052,124.054,123.871,123.979,124.040,124.044,123.859,123.967,124.063,124.067,123.883,123.991,True,-1,124.760825,32.969773,-1,123.8575,124.040


In [21]:
df_plot = df_slim.iloc[70:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x=trades.sTime,
        y=trades[markers[i]],
        mode='markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=250)

In [22]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name

        if row.SIGNAL == BUY:
            self.start_price = row.ask_c
            self.trigger_price = row.ask_c
        else:
            self.start_price = row.bid_c
            self.trigger_price = row.bid_c

        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0

    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price

    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)

In [23]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]

    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [24]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [25]:
df_results.result.sum()

-603.0

In [26]:
df_results.sort_values(by="start_index", inplace=True)

In [27]:
df_m5 = pd.read_csv("../data/candles/EUR_JPY_M5.csv")

In [28]:
df_m5.shape

(812289, 15)

In [29]:
df_m5_slim = df_m5[['time', 'bid_h', 'bid_l', 'ask_h', 'ask_l']].copy()

In [30]:
df_m5_slim.head()

,time,bid_h,bid_l,ask_h,ask_l
0,2013-01-07 00:00:00+00:00,115.272,115.068,115.292,115.090
1,2013-01-07 00:05:00+00:00,115.203,115.074,115.226,115.099
2,2013-01-07 00:10:00+00:00,115.138,115.021,115.157,115.040
3,2013-01-07 00:15:00+00:00,115.141,115.072,115.161,115.099
4,2013-01-07 00:20:00+00:00,115.117,114.862,115.133,114.885


In [31]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [32]:
df_signals['time'] = pd.to_datetime(df_signals['time'])
df_signals['m5_start'] = df_signals['time'] + pd.Timedelta(hours=1)

In [33]:
df_signals['start_index_h1'] = df_signals.index

In [34]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,...,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL,m5_start,start_index_h1
4,2013-01-17 08:00:00+00:00,117.774,118.610,117.761,118.545,117.764,118.602,117.751,118.536,117.784,...,118.554,True,1,117.522991,60.945498,1,119.7090,117.784,2013-01-17 09:00:00+00:00,4
11,2013-01-17 15:00:00+00:00,119.302,119.458,119.146,119.428,119.279,119.449,119.126,119.416,119.326,...,119.439,True,1,117.661294,70.260116,1,119.6085,119.326,2013-01-17 16:00:00+00:00,11
13,2013-01-17 17:00:00+00:00,119.420,120.034,119.418,119.882,119.409,120.024,119.407,119.870,119.432,...,119.893,True,1,117.706077,92.300319,1,120.5845,119.432,2013-01-17 18:00:00+00:00,13
23,2013-01-18 03:00:00+00:00,120.084,120.460,120.078,120.434,120.077,120.455,120.071,120.424,120.091,...,120.443,True,1,117.975841,73.264033,1,120.9710,120.091,2013-01-18 04:00:00+00:00,23
45,2013-01-20 22:00:00+00:00,119.925,120.240,119.910,119.994,119.909,120.226,119.891,119.978,119.941,...,120.011,True,1,118.415687,52.622378,1,120.1160,119.941,2013-01-20 23:00:00+00:00,45


In [35]:
df_signals.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h', 'bid_l',
       'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'ENGULFING', 'direction',
       'EMA_200', 'RSI_14', 'SIGNAL', 'TP', 'SL', 'm5_start',
       'start_index_h1'],
      dtype='object')

In [36]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h', 'bid_l',
                 'ask_o', 'ask_h', 'ask_l', 'ENGULFING', 'direction',
                 'EMA_200', 'RSI_14'], axis=1, inplace=True)

In [37]:
df_signals.head()

,bid_c,ask_c,SIGNAL,TP,SL,m5_start,start_index_h1
4,118.536,118.554,1,119.7090,117.784,2013-01-17 09:00:00+00:00,4
11,119.416,119.439,1,119.6085,119.326,2013-01-17 16:00:00+00:00,11
13,119.870,119.893,1,120.5845,119.432,2013-01-17 18:00:00+00:00,13
23,120.424,120.443,1,120.9710,120.091,2013-01-18 04:00:00+00:00,23
45,119.978,120.011,1,120.1160,119.941,2013-01-20 23:00:00+00:00,45


In [38]:
df_signals.rename(columns={
    'bid_c': 'start_price_BUY',
    'ask_c': 'start_price_SELL',
    'm5_start': 'time'
}, inplace=True)

In [39]:
df_signals.head(2)

,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,time,start_index_h1
4,118.536,118.554,1,119.7090,117.784,2013-01-17 09:00:00+00:00,4
11,119.416,119.439,1,119.6085,119.326,2013-01-17 16:00:00+00:00,11


In [40]:
df_m5_slim.head(2)

,time,bid_h,bid_l,ask_h,ask_l
0,2013-01-07 00:00:00+00:00,115.272,115.068,115.292,115.090
1,2013-01-07 00:05:00+00:00,115.203,115.074,115.226,115.099


In [41]:
df_m5_slim['time'] = pd.to_datetime(df_m5_slim['time'])
df_signals['time'] = pd.to_datetime(df_signals['time'])
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [42]:
merged.fillna(0, inplace=True)

In [43]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [44]:
merged.head()

,time,bid_h,bid_l,ask_h,ask_l,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,start_index_h1
0,2013-01-07 00:00:00+00:00,115.272,115.068,115.292,115.090,0.0,0.0,0,0.0,0.0,0
1,2013-01-07 00:05:00+00:00,115.203,115.074,115.226,115.099,0.0,0.0,0,0.0,0.0,0
2,2013-01-07 00:10:00+00:00,115.138,115.021,115.157,115.040,0.0,0.0,0,0.0,0.0,0
3,2013-01-07 00:15:00+00:00,115.141,115.072,115.161,115.099,0.0,0.0,0,0.0,0.0,0
4,2013-01-07 00:20:00+00:00,115.117,114.862,115.133,114.885,0.0,0.0,0,0.0,0.0,0


In [45]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1

        if row.SIGNAL == BUY:
            self.start_price = row.start_price_BUY
            self.trigger_price = row.start_price_BUY
        else:
            self.start_price = row.start_price_SELL
            self.trigger_price = row.start_price_SELL

        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1

    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price

    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)

In [46]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():

    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))

    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

In [47]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [48]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2437,4,118.536,119.747,1,119.7090,117.784,1.5,2013-01-17 13:30:00+00:00,2013-01-17 09:00:00+00:00,56
1,False,2521,11,119.416,119.899,1,119.6085,119.326,1.5,2013-01-17 17:25:00+00:00,2013-01-17 16:00:00+00:00,19
2,False,2545,13,119.870,120.593,1,120.5845,119.432,1.5,2013-01-17 19:55:00+00:00,2013-01-17 18:00:00+00:00,25
3,False,2665,23,120.424,119.949,1,120.9710,120.091,-1.0,2013-01-18 09:10:00+00:00,2013-01-18 04:00:00+00:00,64
4,False,2920,45,119.978,120.126,1,120.1160,119.941,1.5,2013-01-20 23:10:00+00:00,2013-01-20 23:00:00+00:00,4


In [49]:
df_res_m5.result.sum()

-1416.5